In [219]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [4]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [5]:
answers = {}

In [6]:
# Some data structures that will be useful

In [7]:
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

In [8]:
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

In [9]:
hoursTrain[0]

('u70666506',
 'g49368897',
 {'userID': 'u70666506',
  'early_access': False,
  'hours': 63.5,
  'hours_transformed': 6.011227255423254,
  'found_funny': 1,
  'text': 'If you want to sit in queue for 10-20min and have 140 ping then this game is perfect for you :)',
  'gameID': 'g49368897',
  'user_id': '76561198030408772',
  'date': '2017-05-20'})

In [10]:
##################################################
# Play prediction                                #
##################################################

In [11]:
# Any other preprocessing...

In [12]:
### Question 1

In [13]:
gameCount = defaultdict(int)
totalPlayed = 0

for user,game,_ in readJSON("train.json.gz"):
  gameCount[game] += 1
  totalPlayed += 1

In [14]:
users = set()
games = set() 

for d in allHours : 
    user_id , game_id , _ = d 
    users.add(user_id)
    games.add(game_id)

In [15]:
gamesPerUser = defaultdict(set)
usersPerGame = defaultdict(set)

for user, game , _ in hoursTrain:
    gamesPerUser[user].add(game)
    usersPerGame[game].add(user)

In [16]:
validation = []

for u, g , _ in hoursValid: 
    validation.append((u,g,1))

In [17]:
for u , g, _ in hoursValid: 
    unplayed = list(games - gamesPerUser[u])
    if unplayed :
        negative = random.choice(unplayed)
        validation.append((u , negative , 0))      

In [18]:
#50th percentile 
mostPopular = [(gameCount[x], x ) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

In [19]:
mostPopular[0]

(1092, 'g10773791')

In [20]:
return1 = set()
count = 0 

for ic, i in mostPopular:
    count += ic 
    return1.add(i)
    if count > totalPlayed //2:
        break

In [21]:
correct_pred = 0
for u, g , playedOrNot in validation :
    predicted = 1 if g in return1 else 0 
    if predicted == playedOrNot:
        correct_pred +=1

In [22]:
accuracy = correct_pred / len(validation)
print(accuracy)

0.6817181718171818


In [23]:
answers['Q1'] = accuracy

In [24]:
assertFloat(answers['Q1'])

In [25]:
### Question 2

In [26]:
# Improved strategy


In [27]:
best_threshold = None
best_acc = 0
return_t = set()

for p in range(1, 100):  # Best percentage
    threshold = totalPlayed * (p / 100)
    count = 0
    
    for ic, i in mostPopular:
        count += ic
        return_t.add(i)
        if count > threshold:
            break

    correct_pred = 0
    for u, g, playedOrNot in validation:
        predicted = 1 if g in return_t else 0
        if predicted == playedOrNot:
            correct_pred += 1

    acc = correct_pred / len(validation)
    if acc > best_acc:
        best_acc = acc
        best_threshold = p

In [28]:
print(best_acc, best_threshold)

0.7015701570157016 64


In [29]:
answers['Q2'] = best_acc, best_threshold

In [30]:
assertFloatList(answers['Q2'], 2)

In [31]:
### Question 3/4

In [32]:
def Jaccard(s1, s2):
    numer = len(set(s1).intersection(set(s2) ))
    denom =len(set(s1).union(set(s2)) )
    return numer/denom

In [121]:
threshold = 0.73
predictions = []

for u, g, _ in validation:
    max_sim = 0
    for game in gamesPerUser[u]:
        if game == g:continue
        if u in users:
            similarity = Jaccard(g, game)
        else: 
            similarity = 0  
        max_sim = max(similarity, max_sim)    
        predicted_played = 1 if max_sim > threshold else 0
    predictions.append(predicted_played)

sum(predictions)


13576

In [122]:
accuracy3 = sum(predictions) /len(validation)
print(accuracy3)

0.6788678867886788


In [156]:
return2 = set()
count = 0 

for ic, i in mostPopular:
    count += ic 
    return2.add(i)
    if count > (totalPlayed * .7):
        break

In [163]:
threshold = 0.45
predictions = []

for u, g, _ in validation:
    max_sim = 0
    for game in gamesPerUser[u]:
        if game == g:continue
        if u in users:
            similarity = Jaccard(g, game)
        else: 
            similarity = 0  
        max_sim = max(similarity, max_sim)    
        predicted_played = 1 if max_sim > threshold and game in return2 else 0
    predictions.append(predicted_played)

sum(predictions)



13622

In [158]:
accuracy4 = sum(predictions) /len(validation)
print(accuracy4)

0.6811681168116812


In [159]:
answers['Q3'] = accuracy3
answers['Q4'] = accuracy4

In [160]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [166]:
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    
    u,g = l.strip().split(',')
    max_sim = 0
    for game in gamesPerUser[u]:
        if game == g:continue
        if u in users:
            similarity = Jaccard(g, game)
        else: 
            similarity = 0  
        max_sim = max(similarity, max_sim)  
        if (max_sim > threshold) and (g in return2):
            predictions.write(u + ',' + g + ",1\n")
        else:
            predictions.write(u + ',' + g + ",0\n")

predictions.close()

In [167]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [ ]:
##################################################
# Hours played prediction                        #
##################################################

In [404]:
trainHours = [r[2]['hours_transformed'] for r in hoursTrain]
globalAverage = sum(trainHours) * 1.0 / len(trainHours)

In [440]:
hoursPerUser = defaultdict(set)
hoursPerItem = defaultdict(set)

for user, game , d in hoursTrain:
    hoursPerUser[user].add((game, d['hours_transformed']))
    hoursPerItem[game].add((user, d['hours_transformed']))

In [441]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [442]:
### Question 6

In [443]:
betaU = {}
betaI = {}
for u in hoursPerUser:
    betaU[u] = 0

for g in hoursPerItem:
    betaI[g] = 0

In [444]:
Ntrain = len(hoursTrain)
nUsers = len(hoursPerUser)
nItems = len(hoursPerItem)
users = list(hoursPerUser.keys() )
items = list(hoursPerItem.keys() )

In [445]:
alpha = globalAverage # Could initialize anywhere, this is a guess

In [446]:
def iterate(lamb):
    for itr in range(1000):
        # Calculate alpha
        alpha_term = globalAverage
        for u, i, r in hoursTrain:
            alpha_term +=  r['hours_transformed'] - (betaU[u] + betaI[i])
        alpha = alpha_term / Ntrain

        # Calculate beta_u
        for u in users:
            beta_u_term = 0.
            for i, r in hoursPerUser[u]:
                beta_u_term += r - (alpha+ betaI[i])
            betaU[u] = beta_u_term / (lamb + len(gamesPerUser[u]) )

        # Calculate beta_i
        for i in items:
            beta_i_term = 0.
            for u, r in hoursPerItem[i]:
                beta_i_term += r - (alpha+ betaU[u])
            betaI[i] = beta_i_term / (lamb + len(usersPerGame[i]) )
        
        current_loss = 0.0
        for u, i, r in hoursTrain:
            current_loss += (r['hours_transformed'] - (alpha + betaU[u] + betaI[i]))**2
        current_loss += lamb * (np.sum(np.square(list(betaU.values()))) + np.sum(np.square(list(betaI.values()))))

        # Compare current loss with previous loss and stop/continue
        if itr > 0 and abs(current_loss - previous_loss) < 1e-5:
            break

        previous_loss = current_loss

    return alpha, betaU, betaI

In [447]:
alpha, beta_u, beta_i = iterate(1)

In [448]:
beta_u_avg = sum(beta_u[u] for u in beta_u) / len(beta_u)
beta_i_avg = sum(beta_i[i] for i in beta_i) / len(beta_i)

y_pred = []
y_true = []
for u, i, r in hoursValid:
    beta_u_val = beta_u.get(u, beta_u_avg)
    beta_i_val = beta_i.get(i, beta_i_avg)
    
    val = alpha + beta_u_val + beta_i_val  # TODO: Handle case when u is not in beta_u or i is not in beta_i
    y_pred.append(val)
    y_true.append(r['hours_transformed'])
 
# Compute MSE
ans = MSE(y_pred, y_true)

In [449]:
validMSE = (ans)

In [450]:
print(validMSE)

3.0073719507152656


In [451]:
answers['Q6'] = validMSE

In [452]:
assertFloat(answers['Q6'])

In [390]:
### Question 7

In [391]:
betaUs = [(betaU[u], u) for u in betaU]
betaIs = [(betaI[i], i) for i in betaI]
betaUs.sort()
betaIs.sort()

print("Maximum betaU = " + str(betaUs[-1][1]) + ' (' + str(betaUs[-1][0]) + ')')
print("Maximum betaI = " + str(betaIs[-1][1]) + ' (' + str(betaIs[-1][0]) + ')')
print("Minimum betaU = " + str(betaUs[0][1]) + ' (' + str(betaUs[0][0]) + ')')
print("Minimum betaI = " + str(betaIs[0][1]) + ' (' + str(betaIs[0][0]) + ')')

Maximum betaU = u60898505 (5.828516390799736)
Maximum betaI = g17604638 (5.513893606177892)
Minimum betaU = u13037838 (-3.0057164038915625)
Minimum betaI = g84397720 (-2.7915332948802485)


In [392]:
answers['Q7'] = [betaUs[-1][0], betaUs[0][0], betaIs[-1][0], betaIs[0][0]]

In [393]:
answers['Q7']

[5.828516390799736,
 -3.0057164038915625,
 5.513893606177892,
 -2.7915332948802485]

In [394]:
assertFloatList(answers['Q7'], 4)

In [ ]:
### Question 8

In [454]:
# Better lambda...
lambda_values = [0.1, 0.5, 1.0, 2.0, 5.0]
best_lambda = None
best_validation_error = float('inf')

for lamb in lambda_values:
    alpha, beta_u, beta_i = iterate(lamb)
    beta_u_avg = sum(beta_u[u] for u in beta_u) / len(beta_u)
    beta_i_avg = sum(beta_i[i] for i in beta_i) / len(beta_i)

    y_pred = []
    y_true = [
    for u, i, r in hoursValid:
        beta_u_val = beta_u.get(u, beta_u_avg)
        beta_i_val = beta_i.get(i, beta_i_avg)

        val = alpha + beta_u_val + beta_i_val  # TODO: Handle case when u is not in beta_u or i is not in beta_i
        y_pred.append(val)
        y_true.append(r['hours_transformed'])
 
    ans = MSE(y_pred, y_true)

    if ans < best_validation_error:
        best_validation_error = ans
        best_lambda = lamb

In [457]:
validMSE = ans

In [458]:
answers['Q8'] = (5.0, validMSE)

In [459]:
assertFloatList(answers['Q8'], 2)

In [461]:
predictions = open("HWpredictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    alpha, beta_u, beta_i = iterate(5.0)
    beta_u_avg = sum(beta_u[u] for u in beta_u) / len(beta_u)
    beta_i_avg = sum(beta_i[i] for i in beta_i) / len(beta_i)

    beta_u_val = beta_u.get(u, beta_u_avg)
    beta_i_val = beta_i.get(i, beta_i_avg)


    _ = predictions.write(u + ',' + g + ',' + str(alpha + beta_u_val + beta_i_val) + '\n')

predictions.close()

In [460]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()

In [ ]:
best_lambda